<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SparkSession" data-toc-modified-id="SparkSession-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SparkSession</a></span></li><li><span><a href="#Register-all-tables-for-sql-queries" data-toc-modified-id="Register-all-tables-for-sql-queries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Register all tables for sql queries</a></span></li><li><span><a href="#Prepare-dataset-for-VW" data-toc-modified-id="Prepare-dataset-for-VW-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prepare dataset for VW</a></span></li><li><span><a href="#Train-VW" data-toc-modified-id="Train-VW-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train VW</a></span></li><li><span><a href="#Homework-2:-Baseline-VW-model" data-toc-modified-id="Homework-2:-Baseline-VW-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Homework 2: Baseline VW model</a></span><ul class="toc-item"><li><span><a href="#Our-Code" data-toc-modified-id="Our-Code-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Our Code</a></span><ul class="toc-item"><li><span><a href="#Create-clicks-table" data-toc-modified-id="Create-clicks-table-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Create clicks table</a></span></li><li><span><a href="#Create-Additional-Features" data-toc-modified-id="Create-Additional-Features-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Create Additional Features</a></span><ul class="toc-item"><li><span><a href="#per-document-id---get-topics" data-toc-modified-id="per-document-id---get-topics-5.1.2.1"><span class="toc-item-num">5.1.2.1&nbsp;&nbsp;</span>per document id - get topics</a></span></li></ul></li><li><span><a href="#Join-All-Features" data-toc-modified-id="Join-All-Features-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Join All Features</a></span></li><li><span><a href="#split-features-to-train-test" data-toc-modified-id="split-features-to-train-test-5.1.4"><span class="toc-item-num">5.1.4&nbsp;&nbsp;</span>split features to train test</a></span></li><li><span><a href="#Train-VW" data-toc-modified-id="Train-VW-5.1.5"><span class="toc-item-num">5.1.5&nbsp;&nbsp;</span>Train VW</a></span></li><li><span><a href="#Predict" data-toc-modified-id="Predict-5.1.6"><span class="toc-item-num">5.1.6&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#format-file-for-Kaggle-submission" data-toc-modified-id="format-file-for-Kaggle-submission-5.1.7"><span class="toc-item-num">5.1.7&nbsp;&nbsp;</span>format file for Kaggle submission</a></span></li><li><span><a href="#Create-submission-file" data-toc-modified-id="Create-submission-file-5.1.8"><span class="toc-item-num">5.1.8&nbsp;&nbsp;</span>Create submission file</a></span></li><li><span><a href="#Add-kaggle's-credentials---Make-sure-to-perform-following-commands-on-master-node" data-toc-modified-id="Add-kaggle's-credentials---Make-sure-to-perform-following-commands-on-master-node-5.1.9"><span class="toc-item-num">5.1.9&nbsp;&nbsp;</span>Add kaggle's credentials - Make sure to perform following commands on master node</a></span></li><li><span><a href="#submit-our-HW-to-Kaggle" data-toc-modified-id="submit-our-HW-to-Kaggle-5.1.10"><span class="toc-item-num">5.1.10&nbsp;&nbsp;</span>submit our HW to Kaggle</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# Checklist:
# AWS emr-5.29.0
# MASTER r5d.8xlarge 1x, no EBS
# CORE r5d.8xlarge 4x, no EBS
# Custom bootstrap action: s3://ydatazian/bootstrap.sh
# Allow ssh in master node security group

In [2]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt

# SparkSession

https://spark.apache.org/docs/2.4.4/api/python/pyspark.html

https://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

In [3]:
import findspark
findspark.init()

import spark_utils
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext("yarn", "My App", conf=spark_utils.get_spark_conf())
se = SparkSession(sc)
spark_utils.print_ui_links()

NameNode: http://ec2-52-73-202-8.compute-1.amazonaws.com:50070
YARN: http://ec2-52-73-202-8.compute-1.amazonaws.com:8088
Spark UI: http://ec2-52-73-202-8.compute-1.amazonaws.com:20888/proxy/application_1589891955781_0001


# Register all tables for sql queries

In [4]:
from IPython.display import display
tables = ["clicks_test", "clicks_train", 
          "documents_categories", "documents_entities", "documents_meta", "documents_topics", 
          "events", "page_views", "page_views_sample", "promoted_content"]
for name in tqdm.tqdm(tables):
    df = se.read.parquet("s3://ydatazian/{}.parquet".format(name))
    df.registerTempTable(name)
    print(name)
    display(df.limit(3).toPandas())

clicks_test


,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754


clicks_train


,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1


documents_categories


...

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


page_views_sample


,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


promoted_content


,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7


# Prepare dataset for VW

We will predict a *click* based on:
- ad_id
- document_id
- campaign_id
- advertiser_id

In [21]:
%%time
se.sql("""
select 
    clicks_train.clicked,
    clicks_train.display_id,
    clicks_train.ad_id,
    promoted_content.document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id
from clicks_train join promoted_content on clicks_train.ad_id = promoted_content.ad_id
""").write.parquet("/train_features.parquet", mode='overwrite')

CPU times: user 2.97 ms, sys: 1.33 ms, total: 4.29 ms
Wall time: 54.2 s


In [22]:
se.read.parquet("/train_features.parquet").show(5)

+-------+----------+------+-----------+-----------+-------------+
|clicked|display_id| ad_id|document_id|campaign_id|advertiser_id|
+-------+----------+------+-----------+-----------+-------------+
|      0|         1| 42337|     938164|       5969|         1499|
|      0|         1|139684|    1085937|      17527|         2563|
|      1|         1|144739|    1337362|      18488|         2909|
|      0|         1|156824|     992370|       7283|         1919|
|      0|         1|279295|    1670176|      27524|         1820|
+-------+----------+------+-----------+-----------+-------------+
only showing top 5 rows



In [23]:
# Format: [Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
# https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format
def vw_row_mapper(row):
    clicked = None
    features = []
    for k, v in row.asDict().items():
        if k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        else:
            features.append(k + "_" + v)
    tag = row.display_id + "_" + row.ad_id
    return "{} {}| {}".format(clicked, tag, " ".join(features))

r = se.read.parquet("/train_features.parquet").take(1)[0]
print(r)
print(vw_row_mapper(r))

Row(clicked='0', display_id='1', ad_id='42337', document_id='938164', campaign_id='5969', advertiser_id='1499')
-1 1_42337| display_id_1 ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499


In [24]:
%%time
! hdfs dfs -rm -r /train_features.txt
(
    se.read.parquet("/train_features.parquet")
    .rdd
    .map(vw_row_mapper)
    .saveAsTextFile("/train_features.txt")
)

Deleted /train_features.txt
CPU times: user 27.4 ms, sys: 19 ms, total: 46.5 ms
Wall time: 2min 43s


In [9]:
# copy file to local master node
! rm /mnt/train.txt
! hdfs dfs -getmerge /train_features.txt /mnt/train.txt
# preview local file
! head -n 5 /mnt/train.txt

rm: cannot remove ‘/mnt/train.txt’: No such file or directory
-1 | ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499
-1 | ad_id_139684 document_id_1085937 campaign_id_17527 advertiser_id_2563
1 | ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909
-1 | ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919
-1 | ad_id_279295 document_id_1670176 campaign_id_27524 advertiser_id_1820


# Train VW
https://vowpalwabbit.org/tutorials/getting_started.html

https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments

In [10]:
! ./vw -d /mnt/train.txt -b 24 -c -k --ftrl --passes 1 -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

final_regressor = model
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = /mnt/train.txt.cache
Reading datafile = /mnt/train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.459683 0.459683      8000000      8000000.0  -1.0000  -1.3025        5
0.455707 0.451731     16000000     16000000.0  -1.0000  -1.4668        5
0.452884 0.447239     24000000     24000000.0  -1.0000  -2.3240        5
0.452193 0.450119     32000000     32000000.0   1.0000  -0.5717        5
0.452457 0.453513     40000000     40000000.0  -1.0000  -1.2560        5
0.451587 0.447235     48000000     48000000.0  -1.0000  -2.6222        5
0.450909 0.446843     56000000     56000000.0  -1.0000  -3.4498        5
0.450235 0.445520     64000000     64000000.0  -1.0000  -1.

In [25]:
# make prediction with VW
! echo "? tag1| ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909" > /mnt/test.txt
! echo "? tag2| ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919" >> /mnt/test.txt
! ./vw -d /mnt/test.txt -i model -t -k -p /mnt/predictions.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! cat /mnt/predictions.txt

only testing
predictions = /mnt/predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features

finished run
number of examples = 2
weighted example sum = 2.000000
weighted label sum = 0.000000
average loss = 5.685139
total feature number = 10
0.318580 tag1
0.036084 tag2


# Homework 2: Baseline VW model

Train a baseline model using the following features:
- **clicked**
- geo_location features (country, state, dma)
- day_of_week (from timestamp, use *date.isoweekday()*)
- ad_id
- campaign_id
- advertiser_id
- ad_document_id
- display_document_id
- platform

Make submission to Kaggle to know your leaderboard score

If you want to create a dev set, make a 90%/10% split of training data by display_id

## Our Code

### Create clicks table

In [ ]:
%%time
! hdfs dfs -rm -r /clicks.parquet
se.sql("""
select *
from 
    (
    select 
        *,
        true as is_train
    from clicks_train
    )
    union
    (
    select 
        *,
        "0" as clicked,
        false as is_train
    from clicks_test
    )
""").write.parquet("/clicks.parquet")
se.read.parquet("/clicks.parquet").registerTempTable("clicks")

In [ ]:
se.table("clicks").show(5)

### Create Additional Features

#### per document id - get topics

In [ ]:
%%time
# join everything
! hdfs dfs -rm -r /features.parquet
se.sql("""select 
                t.document_id,
                distinct(topic_id) as topics
                from documents_topics as t
                where confidence_level > 0.5
                """).write.parquet("/display_document_topics.parquet")
se.read.parquet("/display_document_topics").registerTempTable("display_document_topics")


In [ ]:
se.table("display_document_topics").show(5)

In [ ]:
se.sql("select count(*) from display_document_topics").show(1)

### Join All Features

In [ ]:
%%time
# join everything
! hdfs dfs -rm -r /features.parquet
se.sql("""select 
                c.clicked,
                split(v.geo_location, '>')[0] as country, 
                split(v.geo_location, '>')[1] as state,
                split(v.geo_location, '>')[2] as DMA,
                dayofweek(to_date(from_unixtime(v.timestamp + 1465876799998,'YYYY-MM-dd'))) as day_of_week,
                c.ad_id,
                p.campaign_id,
                p.advertiser_id ,
                p.document_id as ad_document_id,
                e.document_id as display_document_id,
                v.platform,
                hour(from_unixtime(v.timestamp + 1465876799998) as hour_in_day,
                m.source_id,
                m.publisher_id,
                dayofweek(to_date(m.publish_time)) as publish_day_of_week,
                hour(to_timestamp(m.publish_time)) as publish_hour,
                t.topics as display_topics
                from clicks as c 
                left join promoted_content as p
                on c.ad_id = p.ad_id
                left join events as e
                on e.display_id = c.display_id
                left join page_views as v
                on v.document_id = e.document_id
                left join document_meta as m
                on m.document_id = e.document_id
                left join display_document_topics as t
                on t.document_id = e.document_id
                """).write.parquet("/features.parquet")
se.read.parquet("/features.parquet").registerTempTable("features")

In [ ]:
se.table("features").show(5)

In [ ]:
# Format: [Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
# https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format
def vw_row_mapper(row):
    clicked = '1' if row.clicked == '1' else '-1'
    # item features
    item_features = "|a "    
    item_features += "ad_id_{}".format(row.ad_id) + " "
    item_features += "campaign_id_{}".format(row.campaign_id) + " "
    item_features += "advertiser_id_{}".format(row.advertiser_id) + " "
    item_features += "ad_document_id_{}".format(ad_document_id) + " "
    
    
    # display features
    display_features = "|d "
    display_features += "display_document_id_{}".format(row.display_document_id) + " "
    display_features += "country_{}".format(row.country) + " "
    display_features += "state_{}".format(row.state) + " "
    display_features += "DMA_{}".format(row.DMA) + " "
    display_features += "day_of_week_{}".format(row.day_of_week) + " "
    display_features += "platform_{}".format(row.platform) + " "
    display_features += "hour_{}".format(row.hour) + " "
    if row.display_topics:
        s = sum(v for k, v in row.display_topics)
        display_features += " ".join("topic{}".format(t) for t in row.display_topics) + " "
    tag = row.display_id + "_" + row.ad_id
    # result
    return clicked + " " + tag + item_features + display_features

r = se.table("features").take(1)[0]

In [ ]:
# Format: [Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
# https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format
def vw_row_mapper(row):
    clicked = None
    features = []
    for k, v in row.asDict().items():
        if k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        elif k =='topics':
            
        else:
            features.append(k + "_" + v)
    tag = row.display_document_id + "_" + row.ad_id
    return "{} {}| {}".format(clicked, tag, " ".join(features))

r = se.read.parquet("/features.parquet").take(1)[0]
print(r)
print(vw_row_mapper(r))

### split features to train test

In [ ]:
%%time
! hdfs dfs -rm -r /train.txt
(
    se.sql("select * from features where is_train = true")
    .rdd
    .map(vw_row_mapper)
    .repartition(16)
    .saveAsTextFile("/train.txt")
)

In [ ]:
%%time
! hdfs dfs -rm -r /test.txt
(
    se.sql("select * from features where is_train = false")
    .rdd
    .map(vw_row_mapper)
    .repartition(16)
    .saveAsTextFile("/test.txt")
)

### Train VW

In [ ]:
%%time
! rm /mnt/train.txt
! hdfs dfs -getmerge /train.txt /mnt/train.txt
! head -n 1 /mnt/train.txt
! du -sh /mnt/train.txt

In [ ]:
! ./vw -d /mnt/train.txt -b 24 -c -k --ftrl --passes 1 -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

### Predict

In [ ]:
! ./vw -d /mnt/test.txt -i model -t -k -p /mnt/predictions.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! head -n 5 /mnt/predictions.txt

In [ ]:
! wc -l /mnt/predictions.txt

### format file for Kaggle submission

In [ ]:
from collections import defaultdict
scores_by_display_id = defaultdict(dict)
for line in tqdm.tqdm(open('/mnt/predictions.txt')):
    score, tag = line.strip().split(" ")
    score = float(score)
    display_id, ad_id = tag.split("_")
    scores_by_display_id[display_id][ad_id] = score

In [ ]:
with open("submission.txt", "w") as f:
    f.write("display_id,ad_id\n")
    for k, vs in tqdm.tqdm_notebook(scores_by_display_id.items()):
        f.write("{},{}\n".format(
            k, 
            " ".join([v[0] for v in sorted(vs.items(), key=lambda x: -x[1])])
        ))

### Create submission file

In [ ]:
with open("submission.txt", "w") as f:
    f.write("display_id,ad_id\n")
    for k, vs in tqdm.tqdm_notebook(scores_by_display_id.items()):
        f.write("{},{}\n".format(
            k, 
            " ".join([v[0] for v in sorted(vs.items(), key=lambda x: -x[1])])
        ))

### Add kaggle's credentials - Make sure to perform following commands on master node

In [ ]:
# mkdir ~/.kaggle
# touch ~/.kaggle/kaggle.json
# echo '{"username":"<your user name>","key":"<your key>"}' > ~/.kaggle/kaggle.json
# cat ~/.kaggle/kaggle.json
# chmod 600 /home/hadoop/.kaggle/kaggle.json

### submit our HW to Kaggle

Obtain Kaggle API token: https://github.com/Kaggle/kaggle-api#api-credentials

Making a submission: https://github.com/Kaggle/kaggle-api#submit-to-a-competition

In [ ]:
! kaggle competitions submit -f submission.txt outbrain-click-prediction -m "baseline"